<a href="https://www.kaggle.com/code/cheikhsadibousidibe/yatc-and-netmanba-finetuning?scriptVersionId=264590814" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import library

In [1]:
!pip install gdown

In [1]:
import os
import pandas as pd

def count_files_in_folder(folder_path):
    return sum(len(files) for _, _, files in os.walk(folder_path))

def get_number_of_classes(train_folder):
    # Number of subfolders = number of classes
    if os.path.exists(train_folder):
        return len([entry for entry in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, entry))])
    return 0

def build_dataset_dataframe(parent_folder, epochs=200, hardware="NVIDIA T4 15GB RAM"):
    rows = []

    for dataset in os.listdir(parent_folder):
        dataset_path = os.path.join(parent_folder, dataset)
        if not os.path.isdir(dataset_path):
            continue

        train_path = os.path.join(dataset_path, "train")
        test_path = os.path.join(dataset_path, "test")

        train_count = count_files_in_folder(train_path)
        test_count = count_files_in_folder(test_path)
        total = train_count + test_count

        if total == 0:
            continue  # avoid divide-by-zero

        train_ratio = int(round((train_count / total) * 100))
        test_ratio = 100 - train_ratio
        split_str = f"{train_ratio}:{test_ratio}"

        num_classes = get_number_of_classes(train_path)

        rows.append({
            "dataset": dataset[:-4],
            "number_of_classes": num_classes,
            "train:test_split": split_str,
            "epochs": epochs,
            "hardware": hardware
        })

    df = pd.DataFrame(rows)
    return df


In [2]:
#yatc
parent_folder = "/kaggle/input/yatc-dataset/YaTC_datasets"
yatc_df = build_dataset_dataframe(parent_folder)
yatc_df=yatc_df.sort_values(by="dataset")
print(yatc_df)

        dataset  number_of_classes train:test_split  epochs  \
0    CICIoT2022                  6            68:32     200   
2   ISCXTor2016                  8            80:20     200   
3   ISCXVPN2016                  7            80:20     200   
1  USTC-TFC2016                 20            80:20     200   

             hardware  
0  NVIDIA T4 15GB RAM  
2  NVIDIA T4 15GB RAM  
3  NVIDIA T4 15GB RAM  
1  NVIDIA T4 15GB RAM  


In [8]:
etbert_df=yatc_df
etbert_df.loc[0,"train:test_split"]="80:20"
etbert_df["epochs"].replace(200,20,inplace=True)
print(etbert_df)

        dataset  number_of_classes train:test_split  epochs  \
0    CICIoT2022                  6            80:20      20   
2   ISCXTor2016                  8            80:20      20   
3   ISCXVPN2016                  7            80:20      20   
1  USTC-TFC2016                 20            80:20      20   

             hardware  
0  NVIDIA T4 15GB RAM  
2  NVIDIA T4 15GB RAM  
3  NVIDIA T4 15GB RAM  
1  NVIDIA T4 15GB RAM  


<ipython-input-8-673dbfce5ca7>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  etbert_df["epochs"].replace(200,20,inplace=True)


In [4]:
import os
import pandas as pd

def count_files_in_folder(folder_path):
    return sum(len(files) for _, _, files in os.walk(folder_path))

def get_number_of_classes(train_folder):
    if os.path.exists(train_folder):
        return len([
            entry for entry in os.listdir(train_folder)
            if os.path.isdir(os.path.join(train_folder, entry))
        ])
    return 0

def build_dataset_summary(root_folder, dataset_name, epochs=120, hardware="NVIDIA T4 15GB RAM"):
    nested_path = os.path.join(root_folder, dataset_name)  # dataset/dataset/
    
    train_path = os.path.join(nested_path, "train")
    test_path = os.path.join(nested_path, "test")

    if not (os.path.exists(train_path) and os.path.exists(test_path)):
        return None

    train_count = count_files_in_folder(train_path)
    test_count = count_files_in_folder(test_path)
    total = train_count + test_count

    if total == 0:
        return None

    train_ratio = int(round((train_count / total) * 100))
    test_ratio = 100 - train_ratio
    split_str = f"{train_ratio}:{test_ratio}"
    num_classes = get_number_of_classes(train_path)

    return {
        "dataset": dataset_name,
        "number_of_classes": num_classes,
        "train:test_split": split_str,
        "epochs": epochs,
        "hardware": hardware
    }

def build_all_dataset_dataframe(parent_folder):
    excluded = {"CrossPlatform-Android", "CrossPlatform-iOS"}
    rows = []

    for dataset_name in os.listdir(parent_folder):
        if dataset_name in excluded:
            continue

        root_folder = os.path.join(parent_folder, dataset_name)
        if not os.path.isdir(root_folder):
            continue

        summary = build_dataset_summary(root_folder, dataset_name)
        if summary:
            rows.append(summary)

    return pd.DataFrame(rows)


In [5]:
# Example usage
parent_folder = "/kaggle/input/netmamba-dataset"
netMamba_df = build_all_dataset_dataframe(parent_folder)
netMamba_df=netMamba_df.sort_values(by="dataset")
print(netMamba_df) 

        dataset  number_of_classes train:test_split  epochs  \
2    CICIoT2022                  6            76:24     120   
0   ISCXTor2016                  8            76:24     120   
3   ISCXVPN2016                  7            76:24     120   
1  USTC-TFC2016                 19            76:24     120   

             hardware  
2  NVIDIA T4 15GB RAM  
0  NVIDIA T4 15GB RAM  
3  NVIDIA T4 15GB RAM  
1  NVIDIA T4 15GB RAM  


In [9]:
#transform to excel sheet
with pd.ExcelWriter('report.xlsx', engine='openpyxl') as writer:
    yatc_df.to_excel(writer, sheet_name='yatc',index=False)
    etbert_df.to_excel(writer, sheet_name='etbert',index=False)
    netMamba_df.to_excel(writer, sheet_name='netMamba',index=False)

In [5]:
#!rm -R /kaggle/working/Miniconda

In [7]:
!rm  /kaggle/working/NetMamba.zip

In [6]:
#!rm -R /kaggle/working/NetMamba


In [5]:

file_id = "1nRz64wljshVVUXhhrS1xye-LxlAIoAvb"
file_name = "NetMamba.zip"  # replace with your actual filename and extension

!gdown --id {file_id} -O {file_name}


'\nfile_id = "1nRz64wljshVVUXhhrS1xye-LxlAIoAvb"\nfile_name = "NetMamba.zip"  # replace with your actual filename and extension\n\n!gdown --id {file_id} -O {file_name}\n'

In [6]:
!unzip NetMamba

In [7]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /Kaggle/working

In [8]:
!chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash Miniconda3-latest-Linux-x86_64.sh -b  -p Miniconda

PREFIX=/kaggle/working/Miniconda
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
  

In [9]:
import os
os.environ['PATH'] = "/kaggle/working/Miniconda/bin:" + os.environ['PATH']
!conda init bash

no change     /kaggle/working/Miniconda/condabin/conda
no change     /kaggle/working/Miniconda/bin/conda
no change     /kaggle/working/Miniconda/bin/conda-env
no change     /kaggle/working/Miniconda/bin/activate
no change     /kaggle/working/Miniconda/bin/deactivate
no change     /kaggle/working/Miniconda/etc/profile.d/conda.sh
no change     /kaggle/working/Miniconda/etc/fish/conf.d/conda.fish
no change     /kaggle/working/Miniconda/shell/condabin/Conda.psm1
no change     /kaggle/working/Miniconda/shell/condabin/conda-hook.ps1
no change     /kaggle/working/Miniconda/lib/python3.13/site-packages/xontrib/conda.xsh
no change     /kaggle/working/Miniconda/etc/profile.d/conda.csh
modified      /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==



In [10]:
!conda create -y -n NetMamba python=3.10.13

Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /kaggle/working/Miniconda/envs/NetMamba

  added / updated specs:
    - python=3.10.13


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pip-25.1                   |     pyhc872135_2         1.3 MB
    python-3.10.13             |       h955ad1f_0        26.8 MB
    setuptools-78.1.1          |  py310h06a4308_0         1.7 MB
    tzdata-2025b               |       h04d1e81_0         116 KB
    wheel-0.45.1               |  py310h06a4308_0         115 KB
    ------------------------------------------------------------
                                           Total:        30.0 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main 
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu 
  bzip2              

In [11]:
#install the correct pytorch version
!conda run -n NetMamba pip install torch==2.1.1 torchvision==0.16.1 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 153.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 125.0 MB/s eta 0:00:00




In [12]:
#install the requirement from NetMamba
!conda run -n NetMamba bash -c "cd NetMamba/mamba-1p1p1 && pip install ."

Processing /kaggle/working/NetMamba/mamba-1p1p1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 34.7 MB/s eta 0:00:00
  Created wheel for mamba_ssm: filename=mamba_ssm-1.1.1-cp310-cp310-linux_x86_64.whl size=137574741 sha256=998cf941c62596ab13766c8144a1a1b269915f501d95e600b28b85627fdf6dee
  Stored in directory: /root/.cache/pip/wheels/95/d5/cd/d52fee1e99d43ae33c3bd18115bdde87e6329660acbdc76f5b
  Created wheel for causal_conv1d: filename=causal_conv1d-1.5.0.post8-cp310-cp310-linux_x86_64.whl size=103717758 sha256=78ed189bdb445f8601e4819e4e0a

In [13]:
!cd NetMamba && conda run -n NetMamba pip install glob2


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9365 sha256=1b9755c05fc2ae372dce2f06c33ce5c4e9349578e7854cc4dfe7fcbcdd7535e8
  Stored in directory: /root/.cache/pip/wheels/37/07/ce/cbe8d31ad93224571b49fa03f8a5da11cdb31d3845ff73e0f3
Successfully built glob2

  DEPRECATION: Building 'glob2' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'glob2'. Discussion can be found at https://github.com/pypa/pip/issues/6334



In [14]:
#install the require librairy
!cd NetMamba && conda run -n NetMamba pip install -r requirements.txt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
!cat /kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py

import contextlib
import functools
import io
import os
import shutil
import subprocess
import sys
import sysconfig

import setuptools


# TODO: is_hip shouldn't be here
def is_hip():
    import torch
    return torch.version.hip is not None


@functools.lru_cache()
def libcuda_dirs():
    libs = subprocess.check_output(["ldconfig", "-p"]).decode()
    # each line looks like the following:
    # libcuda.so.1 (libc6,x86-64) => /lib/x86_64-linux-gnu/libcuda.so.1
    locs = [line.split()[-1] for line in libs.splitlines() if "libcuda.so" in line]
    dirs = [os.path.dirname(loc) for loc in locs]
    msg = 'libcuda.so cannot found!\n'
    if locs:
        msg += 'Possible files are located at %s.' % str(locs)
        msg += 'Please create a symlink of libcuda.so to any of the file.'
    assert any(os.path.exists(os.path.join(path, 'libcuda.so')) for path in dirs), msg
    return dirs


@functools.lru_cache()
def rocm_path_dir():
    return os.getenv("ROCM_PATH", default="/opt/rocm")


@conte

In [16]:
import subprocess

def libcuda_dirs():
    libs = subprocess.check_output(["ldconfig", "-p"]).decode()
    locs = [line.split()[-1] for line in libs.splitlines() if "libcuda.so" in line]
    dirs = [os.path.dirname(loc) for loc in locs]
    if not dirs:
        # Fallback: common locations in Kaggle
        possible_dirs = [
            "/usr/lib/wsl/lib",  # sometimes appears in WSL containers
            "/usr/lib/x86_64-linux-gnu",  # common default
            "/usr/local/cuda/lib64/stubs",  # fallback like in Colab
            "/usr/lib/nvidia",  # sometimes used by drivers
        ]
        for path in possible_dirs:
            if os.path.exists(path):
                dirs.append(path)
    if not dirs:
        raise FileNotFoundError("libcuda.so cannot be found!")
    return dirs
print(libcuda_dirs())


['/usr/lib/x86_64-linux-gnu', '/usr/local/cuda/lib64/stubs']


In [17]:
import torch
print(torch.__version__)

2.5.1+cu121


In [18]:
!python --version

Python 3.13.2


# YATC

In [19]:
#!rm -rf /kaggle/working/logs_dir


!python /kaggle/input/yatc-model-modifie/YaTCKaggle/fine-tune.py \
            --finetune /kaggle/input/pretrained-model/pretrain_model/YaTC_pretrained_model.pth \
            --device cuda --blr 2e-3 --epochs 200 \
            --data_path /kaggle/input/yatc-dataset/yatc_og/ISCXTor2016_MFR --nb_classes 8 \
            --batch_size 64 \
            --output_dir output_dir/yatc

In [ ]:

!python /kaggle/input/yatc-model-modifie/YaTCKaggle/fine-tune.py \
            --finetune /kaggle/input/pretrained-model/pretrain_model/YaTC_pretrained_model.pth \
            --device cuda --blr 2e-3 --epochs 200 \
            --data_path /kaggle/input/yatc-dataset/YaTC_datasets/CICIoT2022_MFR --nb_classes 6 \
            --batch_size 64 \
            --output_dir output_dir/yatc


2025-05-24 07:53:01.745398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-24 07:53:01.995168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-24 07:53:02.063521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Not using distributed mode
[07:53:19.501580] job dir: /kaggle/input/yatc-model-modifie/YaTCKaggle
[07:53:19.501683] Namespace(batch_size=64,
epochs=200,
accum_iter=1,
model='TraFormer_YaTC',
input_size=40,
drop_path=0.1,
clip_grad=None,
weight_decay=0.05,
lr=None,
blr=0.002,
layer_decay=0.75,
min_lr=1e-06,
warmup_epochs=20,
color_jitter=None,
aa='rand-m9-mstd0.5-

In [1]:
!python /kaggle/input/yatc-model-modifie/YaTCKaggle/fine-tune.py \
            --finetune  \/kaggle/working/output_dir/yatc/finetuned_yatc_USTC-TFC2016_MFR.pth \
            --device cuda --blr 2e-3 --epochs 20\
            --data_path /kaggle/input/yatc-dataset/YaTC_datasets/USTC-TFC2016_MFR --nb_classes 20 \
            --batch_size 64 \
            --output_dir output_dir/yatc

2025-05-25 15:53:46.166993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-25 15:53:46.396273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-25 15:53:46.458731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Not using distributed mode
[15:54:03.393894] job dir: /kaggle/input/yatc-model-modifie/YaTCKaggle
[15:54:03.393982] Namespace(batch_size=64,
epochs=20,
accum_iter=1,
model='TraFormer_YaTC',
input_size=40,
drop_path=0.1,
clip_grad=None,
weight_decay=0.05,
lr=None,
blr=0.002,
layer_decay=0.75,
min_lr=1e-06,
warmup_epochs=20,
color_jitter=None,
aa='rand-m9-mstd0.5-i

In [23]:
!python /kaggle/input/yatc-model-modifie/YaTCKaggle/fine-tune.py \
            --finetune /kaggle/input/pretrained-model/pretrain_model/YaTC_pretrained_model.pth \
            --device cuda --blr 2e-3 --epochs 15 \
            --data_path /kaggle/input/yatc-dataset/yatc_og/ISCXVPN2016_MFR --nb_classes 7 \
            --batch_size 64 \
            --output_dir output_dir/yatc


'!python /kaggle/input/yatc-model-modifie/YaTCKaggle/fine-tune.py             --finetune /kaggle/input/pretrained-model/pretrain_model/YaTC_pretrained_model.pth             --device cuda --blr 2e-3 --epochs 200             --data_path /kaggle/input/yatc-dataset/yatc_og/ISCXVPN2016_MFR --nb_classes 7             --batch_size 64             --output_dir output_dir/yatc\n'

## Evaluation of yatc models

In [5]:

import sys
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SequentialSampler

# Add the YaTC module path to import engine and models
sys.path.append("/kaggle/input/yatc-model-modifie/YaTCKaggle")

import math
import sys
from typing import Iterable, Optional

import torch

from timm.data import Mixup
from timm.utils import accuracy

import util.misc as misc
import util.lr_sched as lr_sched

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

import matplotlib.pyplot as plt
import numpy as np

from engine import evaluate
import models_YaTC
from util.pos_embed import interpolate_pos_embed
import numpy as np
import util.misc as misc
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    classification_report
)


In [6]:
@torch.inference_mode()
def evaluate1(data_loader, model, device):
    criterion = torch.nn.CrossEntropyLoss()

    metric_logger = misc.MetricLogger(delimiter="  ")
    header = 'Test:'

    # switch to evaluation mode
    model.eval()

    pred_all = []
    target_all = []

    for batch in metric_logger.log_every(data_loader, 10, header):
        images = batch[0]
        target = batch[-1]
        images = images.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        # compute output
        with torch.amp.autocast("cuda"):
            output = model(images)
            loss = criterion(output, target)

        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()

        pred_all.extend(pred[0].cpu())
        target_all.extend(target.cpu())

        acc1, acc5 = accuracy(output, target, topk=(1, 5))

        batch_size = images.shape[0]
        metric_logger.update(loss=loss.item())
        metric_logger.meters['acc1'].update(acc1.item()/100, n=batch_size)
        metric_logger.meters['acc5'].update(acc5.item()/100, n=batch_size)

    macro = precision_recall_fscore_support(target_all, pred_all, average='weighted')
    cm = confusion_matrix(target_all, pred_all)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print('* Acc@1 {top1.global_avg:.4f} Acc@5 {top5.global_avg:.4f} loss {losses.global_avg:.4f}'
          .format(top1=metric_logger.acc1, top5=metric_logger.acc5, losses=metric_logger.loss))
    print(
        '* Pre {macro_pre:.4f} Rec {macro_rec:.4f} F1 {macro_f1:.4f}'
        .format(macro_pre=macro[0], macro_rec=macro[1],
                    macro_f1=macro[2]))
        
    test_state = {k: meter.global_avg for k, meter in metric_logger.meters.items()}
    test_state['loss'] = test_state.pop('loss')  # already present
    test_state['acc'] = test_state.pop('acc1')   # rename acc1 → acc
    test_state['weighted_pre'] = macro[0]
    test_state['weighted_rec'] = macro[1]
    test_state['weighted_f1'] = macro[2]
    test_state['cm'] = cm.tolist()  # convert np.array to list

    return test_state

In [7]:
import json
import os

def evaluation(
    model_path='',
    data_path='',
    batch_size=64,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    output_json_path=None , # <-- new optional parameter
    dirs="/kaggle/working/"
):
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader, SequentialSampler
    from engine import evaluate  # <- assumes your `evaluate()` is in engine.py

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])

    test_dir = os.path.join(data_path, 'test')
    test_dataset = datasets.ImageFolder(test_dir, transform=transform)
    test_loader = DataLoader(
        test_dataset,
        sampler=SequentialSampler(test_dataset),
        batch_size=batch_size,
        num_workers=2,
        pin_memory=True,
        drop_last=False
    )

    model = torch.load(model_path, map_location=device)
    model.to(device)
    model.eval()

    test_stats = evaluate1(test_loader, model, device)

    print(f"✅ Accuracy: {test_stats['acc']:.4f}")
    print(f"✅ Weighted F1 Score: {test_stats['weighted_f1']:.4f}")

    # Write to JSON
    os.makedirs(dirs,exist_ok=True)
    if output_json_path:
        with open(dirs+output_json_path, 'w') as f:
            json.dump(test_stats, f, indent=2)
        print(f"📁 Results saved to: {output_json_path}")

    return test_stats


In [8]:
result= evaluation(
    model_path='/kaggle/working/output_dir/yatc/finetuned_yatc_CICIoT2022_MFR.pth',
    data_path='/kaggle/input/yatc-dataset/YaTC_datasets/CICIoT2022_MFR',
    batch_size=64,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    output_json_path="Result_yatc_CICIoT2022.json",
    dirs="result/yatc/"
)

KeyboardInterrupt: 

In [9]:
result= evaluation(
    model_path='/kaggle/working/output_dir/yatc/finetuned_yatc_USTC-TFC2016_MFR.pth',
    data_path='/kaggle/input/yatc-dataset/YaTC_datasets/USTC-TFC2016_MFR',
    batch_size=64,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    output_json_path="Result_yatc_USTC-TFC2016.json",
    dirs="result/yatc/"
)

<ipython-input-7-6117e767ebd2>:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Input shape at start: torch.Size([64, 1, 40, 40])
Shape after collecting features: torch.Size([64, 105, 192])
Shape after pooling: torch.Size([64, 192])
Shape after norm: torch.Size([64, 192])
Final output shape: torch.Size([64, 20])
Test:  [ 0/16]  loss: 0.0951 (0.0951)  acc1: 1.0000 (1.0000)  acc5: 1.0000 (1.0000)  eta: 0:00:10  time: 0.6756  data: 0.1562  max mem: 136
Input shape at start: torch.Size([64, 1, 40, 40])
Shape after collecting features: torch.Size([64, 105, 192])
Shape after pooling: torch.Size([64, 192])
Shape after norm: torch.Size([64, 192])
Final output shape: torch.Size([64, 20])
Input shape at start: torch.Size([64, 1, 40, 40])
Shape after collecting features: torch.Size([64, 105, 192])
Shape after pooling: torch.Size([64, 192])
Shape after norm: torch.Size([64, 192])
Final output shape: torch.Size([64, 20])
Input shape at start: torch.Size([64, 1, 40, 40])
Shape after collecting features: torch.Size([64, 105, 192])
Shape after pooling: torch.Size([64, 192])
Shape

# NetMamba

**Here for the netMamba we found out that we need a conda env in order to run the code** 

In [27]:
file_path = '/kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py'


with open(file_path, 'r') as f:
    lines = f.readlines()

# Keep all lines except the one you added
with open(file_path, 'w') as f:
    for line in lines:
        if 'dirs.append("/usr/local/cuda/cuda/lib64/stubs")' not in line:
            f.write(line)


In [28]:
file_path = '/kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py'

with open(file_path, 'r') as f:
    lines = f.readlines()

# Insert your custom line after line 10 (index 10)
lines.insert(26, '    dirs.append("/usr/local/cuda/lib64/stubs")\n')

with open(file_path, 'w') as f:
    f.writelines(lines)


In [29]:

file_path = '/kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py'

with open(file_path, 'r') as f:
    lines = f.readlines()

# Insert your custom line after line 10 (index 10)
lines.insert(26, '    dirs.append("/usr/lib/x86_64-linux-gnu")\n')

with open(file_path, 'w') as f:
    f.writelines(lines)


In [30]:

file_path = '/kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py'

with open(file_path, 'r') as f:
    lines = f.readlines()

# Insert your custom line after line 10 (index 10)
lines.insert(1, 'import glob\n')

with open(file_path, 'w') as f:
    f.writelines(lines)


In [31]:
import re

file_path = '/kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py'

new_function = '''
@functools.lru_cache()
def libcuda_dirs():
    # On Kaggle, we'll provide the paths directly instead of checking
    # This is more permissive and will work even if libcuda.so doesn't exist yet
    dirs = ['/usr/lib/x86_64-linux-gnu', '/usr/local/cuda/lib64/stubs']
    # Skip the existence check for Kaggle
    return dirs
'''

# Read current contents
with open(file_path, 'r') as f:
    contents = f.read()

# Replace old libcuda_dirs function
new_contents = re.sub(
    r'@functools\.lru_cache\(\)\s+def libcuda_dirs\([\s\S]*?return dirs',
    new_function.strip(),
    contents
)

# Write the modified contents back
with open(file_path, 'w') as f:
    f.write(new_contents)

print("✅ libcuda_dirs() function successfully replaced.")


✅ libcuda_dirs() function successfully replaced.


In [32]:
# Step 1: Create symbolic links from the existing libraries to where our build script expects them

# First, let's add the actual CUDA library paths to our environment
import os

# These are the paths where we found libcuda.so
cuda_lib_paths = [
    '/usr/local/nvidia/lib64',
    '/usr/local/cuda-12.2/compat'
]

# Update environment variables to include these paths
os.environ['LD_LIBRARY_PATH'] = ':'.join(cuda_lib_paths + [os.environ.get('LD_LIBRARY_PATH', '')])
os.environ['LIBRARY_PATH'] = ':'.join(cuda_lib_paths + [os.environ.get('LIBRARY_PATH', '')])

# Step 2: Create a modified build script that uses these actual libraries

In [33]:
file_path = '/kaggle/working/Miniconda/envs/NetMamba/lib/python3.10/site-packages/triton/common/build.py'

new_contents = '''
import contextlib
import functools
import io
import os
import shutil
import subprocess
import sys
import sysconfig

import setuptools


# TODO: is_hip shouldn't be here
def is_hip():
    import torch
    return torch.version.hip is not None


@functools.lru_cache()
def libcuda_dirs():
    # Use the actual locations where libcuda.so was found
    return [
        '/usr/local/nvidia/lib64',
        '/usr/local/cuda-12.2/compat',
        '/usr/lib/x86_64-linux-gnu'
    ]


@functools.lru_cache()
def rocm_path_dir():
    return os.getenv("ROCM_PATH", default="/opt/rocm")


@contextlib.contextmanager
def quiet():
    old_stdout, old_stderr = sys.stdout, sys.stderr
    sys.stdout, sys.stderr = io.StringIO(), io.StringIO()
    try:
        yield
    finally:
        sys.stdout, sys.stderr = old_stdout, old_stderr


@functools.lru_cache()
def cuda_include_dir():
    # Try to use actual CUDA include directories that exist in the environment
    if os.path.exists('/usr/local/cuda-12.2/include'):
        return '/usr/local/cuda-12.2/include'
    
    # Fallback to the original path
    base_dir = os.path.join(os.path.dirname(__file__), os.path.pardir)
    cuda_path = os.path.join(base_dir, "third_party", "cuda")
    return os.path.join(cuda_path, "include")


def _build(name, src, srcdir):
    if is_hip():
        hip_lib_dir = os.path.join(rocm_path_dir(), "lib")
        hip_include_dir = os.path.join(rocm_path_dir(), "include")
    else:
        cuda_lib_dirs = libcuda_dirs()
        cu_include_dir = cuda_include_dir()
    
    suffix = sysconfig.get_config_var('EXT_SUFFIX')
    so = os.path.join(srcdir, '{name}{suffix}'.format(name=name, suffix=suffix))
    
    # try to avoid setuptools if possible
    cc = os.environ.get("CC")
    if cc is None:
        # TODO: support more things here.
        clang = shutil.which("clang")
        gcc = shutil.which("gcc")
        cc = gcc if gcc is not None else clang
        if cc is None:
            raise RuntimeError("Failed to find C compiler. Please specify via CC environment variable.")
    
    # This function was renamed and made public in Python 3.10
    if hasattr(sysconfig, 'get_default_scheme'):
        scheme = sysconfig.get_default_scheme()
    else:
        scheme = sysconfig._get_default_scheme()
    
    # 'posix_local' is a custom scheme on Debian. However, starting Python 3.10, the default install
    # path changes to include 'local'. This change is required to use triton with system-wide python.
    if scheme == 'posix_local':
        scheme = 'posix_prefix'
    py_include_dir = sysconfig.get_paths(scheme=scheme)["include"]

    if is_hip():
        ret = subprocess.check_call([cc, src, f"-I{hip_include_dir}", f"-I{py_include_dir}", 
                                     f"-I{srcdir}", "-shared", "-fPIC", f"-L{hip_lib_dir}", 
                                     "-lamdhip64", "-o", so])
    else:
        # Set environment variables for the subprocess
        env = os.environ.copy()
        env['LD_LIBRARY_PATH'] = ':'.join(cuda_lib_dirs) + ':' + env.get('LD_LIBRARY_PATH', '')
        
        # Try several compilation approaches
        success = False
        
        # Approach 1: Standard compilation with explicit libcuda
        try:
            cc_cmd = [cc, src, "-O3", f"-I{cu_include_dir}", f"-I{py_include_dir}", 
                      f"-I{srcdir}", "-shared", "-fPIC", "-o", so]
            cc_cmd += [f"-L{dir}" for dir in cuda_lib_dirs]
            cc_cmd += ["-lcuda"]
            subprocess.check_call(cc_cmd, env=env)
            success = True
        except subprocess.CalledProcessError:
            print("Warning: First compilation approach failed, trying another...")
        
        # Approach 2: Try with full path to libcuda.so
        if not success:
            try:
                for lib_dir in cuda_lib_dirs:
                    if os.path.exists(os.path.join(lib_dir, 'libcuda.so')):
                        lib_path = os.path.join(lib_dir, 'libcuda.so')
                        cc_cmd = [cc, src, "-O3", f"-I{cu_include_dir}", f"-I{py_include_dir}", 
                                f"-I{srcdir}", "-shared", "-fPIC", "-o", so, lib_path]
                        cc_cmd += [f"-L{dir}" for dir in cuda_lib_dirs]
                        subprocess.check_call(cc_cmd, env=env)
                        success = True
                        break
            except subprocess.CalledProcessError:
                print("Warning: Second compilation approach failed, trying another...")
        
        # Approach 3: Try without explicit cuda linking
        if not success:
            try:
                cc_cmd = [cc, src, "-O3", f"-I{cu_include_dir}", f"-I{py_include_dir}", 
                        f"-I{srcdir}", "-shared", "-fPIC", "-o", so]
                cc_cmd += [f"-L{dir}" for dir in cuda_lib_dirs]
                subprocess.check_call(cc_cmd, env=env)
                success = True
            except subprocess.CalledProcessError:
                print("Warning: Third compilation approach failed, falling back to setuptools...")
        
        ret = 0 if success else 1

    if ret == 0:
        return so
    
    # fallback on setuptools
    extra_compile_args = ['-O3']
    library_dirs = cuda_lib_dirs
    include_dirs = [srcdir, cu_include_dir, py_include_dir]
    
    # Try both with and without cuda in libraries
    libraries = []  # First try without cuda
    
    try:
        # Create extension module
        ext = setuptools.Extension(
            name=name,
            language='c',
            sources=[src],
            include_dirs=include_dirs,
            extra_compile_args=extra_compile_args,
            library_dirs=library_dirs,
            libraries=libraries,
        )
        
        # Build extension module
        args = ['build_ext']
        args.append('--build-temp=' + srcdir)
        args.append('--build-lib=' + srcdir)
        args.append('-q')
        args = dict(
            name=name,
            ext_modules=[ext],
            script_args=args,
        )
        
        with quiet():
            setuptools.setup(**args)
    except:
        # Try again with cuda in libraries
        libraries = ['cuda']
        
        # Create extension module
        ext = setuptools.Extension(
            name=name,
            language='c',
            sources=[src],
            include_dirs=include_dirs,
            extra_compile_args=extra_compile_args,
            library_dirs=library_dirs,
            libraries=libraries,
        )
        
        # Build extension module
        args = ['build_ext']
        args.append('--build-temp=' + srcdir)
        args.append('--build-lib=' + srcdir)
        args.append('-q')
        args = dict(
            name=name,
            ext_modules=[ext],
            script_args=args,
        )
        
        with quiet():
            setuptools.setup(**args)
    
    return so
'''.strip()

# Write the new contents, overwriting the file
with open(file_path, 'w') as f:
    f.write(new_contents)

print("✅ File content successfully replaced.")


✅ File content successfully replaced.


In [34]:
!conda run -n NetMamba python3.10 NetMamba/src/fine-tune.py \
    --blr 2e-3 \
    --epochs 120 \
    --nb_classes 7 \
    --finetune /kaggle/input/pretrained-model/pretrain_model/netMamba_pretrained_model.pth \
    --data_path /kaggle/input/netmamba-dataset/ISCXVPN2016/ISCXVPN2016 \
    --output_dir output_dir/NetMamba/ISCXVPN2016 \
    --log_dir logs_dir/NetMamba/ISCXVPN2016 \
    --model net_mamba_classifier \
    --num_workers 4 \
    --no_amp

Not using distributed mode
[13:49:17.870878] job dir: /kaggle/working/NetMamba/src
[13:49:17.870957] Namespace(batch_size=64,
epochs=120,
save_steps_freq=5000,
accum_iter=1,
model='net_mamba_classifier',
input_size=40,
drop_path=0.1,
clip_grad=None,
weight_decay=0.05,
lr=None,
blr=0.002,
layer_decay=0.75,
min_lr=1e-06,
warmup_epochs=20,
color_jitter=None,
aa='rand-m9-mstd0.5-inc1',
smoothing=0.1,
reprob=0.25,
remode='pixel',
recount=1,
resplit=False,
mixup=0,
cutmix=0,
cutmix_minmax=None,
mixup_prob=1.0,
mixup_switch_prob=0.5,
mixup_mode='batch',
finetune='/kaggle/input/pretrained-model/pretrain_model/netMamba_pretrained_model.pth',
data_path='/kaggle/input/netmamba-dataset/ISCXVPN2016',
nb_classes=7,
output_dir='output_dir/NetMamba/ISCXVPN2016',
log_dir='logs_dir/NetMamba/ISCXVPN2016',
device='cuda',
seed=0,
resume='',
start_epoch=0,
eval=False,
dist_eval=False,
num_workers=4,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
distributed=False,
dist_url='env://',
if_amp=Fa

In [ ]:
!conda run -n NetMamba python3.10 NetMamba/src/fine-tune.py \
    --blr 2e-3 \
    --epochs 120 \
    --nb_classes 20 \
    --finetune /kaggle/input/pretrained-model/pretrain_model/netMamba_pretrained_model.pth \
    --data_path /kaggle/input/netmamba-dataset/USTC-TFC2016/USTC-TFC2016 \
    --output_dir output_dir/NetMamba/USTC_TFC2016 \
    --log_dir logs_dir/NetMamba/USTC_TFC2016 \
    --model net_mamba_classifier \
    --num_workers 4 \
    --no_amp

In [ ]:
!conda run -n NetMamba python3.10 NetMamba/src/fine-tune.py \
    --blr 2e-3 \
    --epochs 120 \
    --nb_classes 8 \
    --finetune /kaggle/input/pretrained-model/pretrain_model/netMamba_pretrained_model.pth \
    --data_path /kaggle/input/netmamba-dataset/ISCXTor2016/ISCXTor2016\
    --output_dir output_dir/NetMamba/ISCXTor2016 \
    --log_dir logs_dir/NetMamba/ISCXTor2016 \
    --model net_mamba_classifier \
    --num_workers 4 \
    --no_amp

In [ ]:
!conda run -n NetMamba python3.10 NetMamba/src/fine-tune.py \
    --blr 2e-3 \
    --epochs 120 \
    --nb_classes 6 \
    --finetune /kaggle/input/pretrained-model/pretrain_model/netMamba_pretrained_model.pth \
    --data_path /kaggle/input/netmamba-dataset/CICIoT2022/CICIoT2022\
    --output_dir output_dir/NetMamba/CICIoT2022 \
    --log_dir logs_dir/NetMamba/CICIoT2022 \
    --model net_mamba_classifier \
    --num_workers 4 \
    --no_amp

In [ ]:
# Check if CUDA is installed and get its version
#!nvcc --version

# Look for CUDA-related directories
#!ls -la /usr/local | grep cuda

# Look for any cuda libraries in standard locations
#!find /usr/ -name "*cuda*.so*" 2>/dev/null

In [ ]:
!rm -R /kaggle/working/Miniconda